# Installs and Imports

In [1]:
# mounting drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/BullingerDigitalLMFootnotes/git_repo/evaluate


/content/drive/MyDrive/BullingerDigitalLMFootnotes/git_repo/evaluate


In [3]:
%pip install evaluate
%pip install transformers
%pip install bert-score
%pip install rouge_score

  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
Using cached bert_score-0.3.13-py3-none-any.whl (61 kB)
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=09cba945dda46d5c2582833d1419b22b88be7bab3c3ce29dfc9a598611e87557
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
from evaluate_utils import *
import pandas as pd
import os
def get_merged_df(human_df, machine_df, filepath):
  if os.path.exists(filepath):
    merged_df = pd.read_csv(filepath)
  else:
    merged_df = machine_df.merge(human_df, on=["letter_id", "n_footnote"])
    merged_df = merged_df[["letter_id", "n_footnote", "generated_footnote", "xml_footnote", "text_footnote"]]
    merged_df.to_csv(filepath)
  return merged_df

pd.set_option('display.max_colwidth', None)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [40]:

human_footnotes_df = pd.read_csv("../../data/footnote_downsized_df.csv")
human_footnotes_df["xml_footnote"] = human_footnotes_df["xml_footnote"].apply(remove_outer_note_tag)

# Evaluate LLama Generation

## Calculate BLEU, ROUGE and BERT-SCORE

In [41]:
generated_footnotes_df = pd.read_csv("../../data/model_responses/llama/Meta-Llama-3.1-8B-Instruct_instruct_add_test.csv")
eval_df_filename = "evaluation-Meta-Llama-3.1-8B-Instruct_instruct_add_test.csv"
merged_df = get_merged_df(human_footnotes_df, generated_footnotes_df, eval_df_filename)
evaluator = genEvaluator(merged_df)

In [24]:
evaluator.compute_bleu_rouge_without_markup()
merged_df = evaluator.update_and_return_df()

calculating BLEU and ROUGE without markup


100%|██████████| 2873/2873 [04:57<00:00,  9.65it/s]


In [25]:
merged_df

,Unnamed: 0.1,Unnamed: 0,letter_id,n_footnote,generated_footnote,xml_footnote,text_footnote,bert_with,bert_without,bleu_with,rouge_with,bleu_without,rouge_without
0,0,0,12891,10,"Die drei Briefe, die ich empfangen habe, sind von dem König von Frankreich, dem König von England und dem König von Spanien.",1mm von einandren gan: Beim Auseinandergehen (nach der Sitzung).,1mm von einandren gan: Beim Auseinandergehen (nach der Sitzung).,0.623506,0.623506,0.000000,0.058824,0.0,0.058824
1,1,1,11456,3,Zu den Ammianischen Geschichten.,"<persName ref=""p8011"" cert=""high"">Gwalthers</persName> Brief an <persName ref=""p167"" cert=""high"">Johann Jakob Ammann</persName> ist nicht erhalten.",Gwalthers Brief an Johann Jakob Ammann ist nicht erhalten.,0.526715,0.644456,0.000000,0.000000,0.0,0.000000
2,2,2,10729,20,"1000 gl. verrösslet und nütt geschaffet, nämlich die Kosten für die Belagerung von <placeName ref=""l167"" cert=""high"">Jenff</placeName>.","<placeName ref=""l1280"" cert=""high"">Thonon (Chablais)</placeName>.",Thonon (Chablais).,0.792157,0.604425,0.370373,0.312500,0.0,0.000000
3,3,3,12453,21,Von der Saxonischen und Lünenburgischen Gesandtschaft ist nichts zu hören.,"<persName ref=""p18509"" cert=""high"">Herzog August von Sachsen</persName>.",Herzog August von Sachsen.,0.559318,0.665829,0.000000,0.090909,0.0,0.125000
4,4,4,12436,3,"Crediderim ergo non inconsultum fore, si ea hora tractarem historicum aliquid ex veteri testamento, vel Genesim vel quid aliud tale, s. <ref target=""file12377"">Nr. 2330</ref>, Anm. 14.",Gemeint ist: historicum librum.,Gemeint ist: historicum librum.,0.603631,0.623745,0.000000,0.060606,0.0,0.068966
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2868,2868,2868,13053,18,Malumus in acie cadere fortiter quam serviliter et turpiter subiacere libidini impii militis.,"Vgl. etwa <bibl>HBBW</bibl> XIX 240,38f. - Hier liegt eine Pointe gegen die Reichsstädte vor. Vgl. auch <ref target=""file13057"">Nr. 3008</ref>,[5].","Vgl. etwa HBBW XIX 240,38f. - Hier liegt eine Pointe gegen die Reichsstädte vor. Vgl. auch Nr. 3008,[5].",0.522203,0.556036,0.000000,0.000000,0.0,0.000000
2869,2869,2869,11455,11,"Cęterum, quid nostri responderint, similiter accipe: „Neminem iustum possessorem unquam deiecimus, et sunt bona ecclesiastica bona ex parte nostra.“ (s. oben <ref target=""file11447"">Nr. 1404</ref>, S. 4).","Vgl. <bibl>ADRG</bibl> I/I 114-128, Nr. 29 (bes. S. 120, 20-127, 37); <bibl>PC</bibl> III 81.","Vgl. ADRG I/I 114-128, Nr. 29 (bes. S. 120, 20-127, 37); PC III 81.",0.589988,0.611891,0.000000,0.078431,0.0,0.093023
2870,2870,2870,11535,2,"""Quorsum enim, mi ter mihi venerande atque charissime Bullingere, attinet tantas pro tantilla re gratias agere, mihi praesertim, qui tot tibi nominibus, quod serio scribo, devinctus sum..."".","Siehe oben <ref target=""file11520"">Nr. 1477</ref>, 2-5.","Siehe oben Nr. 1477, 2-5.",0.507285,0.517042,0.000000,0.000000,0.0,0.000000
2871,2871,2871,12823,28,"Ich bin gar angsthafft in der sach, weil der Kurfürst von Augsburg und der Landgraf von Hessen sich rüsten, um gegen die Eidgenossen und den Franzosen zu kämpfen, und ich fürchte, dass dies zu einem blutigen Konflikt führen wird.",besorgt; s. <hi>FNHDW</hi> I 1197f.,besorgt; s. FNHDW I 1197f.,0.528882,0.550350,0.000000,0.000000,0.0,0.000000


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [44]:
bleu_with_sort = merged_df.sort_values(by=["bleu_with"])
bleu_with_sort.round(2)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,letter_id,n_footnote,generated_footnote,xml_footnote,text_footnote,bert_with,bert_without,bleu_with,rouge_with,bleu_without,rouge_without
0,0,0,0,12891,10,"Die drei Briefe, die ich empfangen habe, sind von dem König von Frankreich, dem König von England und dem König von Spanien.",1mm von einandren gan: Beim Auseinandergehen (nach der Sitzung).,1mm von einandren gan: Beim Auseinandergehen (nach der Sitzung).,0.62,0.62,0.00,0.06,0.00,0.06
1775,1775,1775,1775,12891,26,Das Verhältnis zwischen dem König Ferdinand und dem Papst ist nicht klar.,"Vgl. <ref target=""file12849"">Nr. 2800</ref>,63-65; <ref target=""file12869"">Nr. 2820</ref>,82-84.","Vgl. Nr. 2800,63-65; Nr. 2820,82-84.",0.50,0.56,0.00,0.00,0.00,0.00
1777,1777,1777,1777,12824,36,"""Der starck gott kan dem schympf bald den boden usstossen.""","Spaß. -Der Ausdruck ""dem schympf den boden usstossen"" bedeutet dem Vergnügen ein Ende machen; vgl. <hi>Kessler, Sabbata</hi> 690.","Spaß. -Der Ausdruck ""dem schympf den boden usstossen"" bedeutet dem Vergnügen ein Ende machen; vgl. Kessler, Sabbata 690.",0.69,0.71,0.00,0.39,0.00,0.41
1779,1779,1779,1779,12756,5,"Der Brief ist an Bullinger gerichtet und ist als Antwort auf zwei vorherige Briefe von Bullinger zu verstehen, die Blarer am 24/25. November und am 1. Dezember geschrieben hat.","<placeName ref=""l2351"" cert=""high"">Attikon</placeName> (Kt. Zürich). - Zur Lage der Ortschaft s. <ref target=""file12682"">Nr. 2635</ref>, Anm. 31.","Attikon (Kt. Zürich). - Zur Lage der Ortschaft s. Nr. 2635, Anm. 31.",0.54,0.58,0.00,0.04,0.00,0.05
1780,1780,1780,1780,10729,29,"Die ""Botten"" sind die Gesandten oder Boten, die von den Bernern geschickt wurden, um mit den Saffoyern zu verhandeln.","<persName ref=""p7451"" cert=""high"">Ludwig von Diesbach</persName> und <persName ref=""p5980"" cert=""high"">Hans Rudolf Nägeli</persName>, vgl. <bibl>EA</bibl> IV/1c 569f.","Ludwig von Diesbach und Hans Rudolf Nägeli, vgl. EA IV/1c 569f.",0.54,0.60,0.00,0.04,0.00,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,2479,2479,2479,12840,20,"20. vgl. <ref target=""file12835"">Nr. 2786</ref>","Vgl. <ref target=""file12835"">Nr. 2786</ref>,39-41.","Vgl. Nr. 2786,39-41.",0.89,0.80,0.72,0.82,0.00,0.67
862,862,862,862,13003,7,"Zum ital. Krieg, s. zuletzt <ref target=""file12992"">Nr. 2943</ref>,<ptr target=""qvzl17208-2"" type=""qv""/>.","Siehe dazu zuletzt <ref target=""file12992"">Nr. 2943</ref>,<ptr target=""qvzl17208-5"" type=""qv""/>.","Siehe dazu zuletzt Nr. 2943,.",0.93,0.78,0.76,0.75,0.38,0.50
774,774,774,774,10154,15,"Vgl. <cit type=""bible""><ref>Ps 137,8-9</ref></cit>.","Vgl. <cit type=""bible""><ref>Mt 10,15</ref></cit>.","Vgl. Mt 10,15.",0.96,0.87,0.77,0.67,0.00,0.22
2414,2414,2414,2414,13003,3,"Zu diesem s. zuletzt <ref target=""file12992"">Nr. 2943</ref>,<ptr target=""qvzl17208-2"" type=""qv""/>.","Bezug auf Myconius' Bitte in <ref target=""file12992"">Nr. 2943</ref>,<ptr target=""qvzl17208-4"" type=""qv""/>.","Bezug auf Myconius' Bitte in Nr. 2943,.",0.94,0.76,0.79,0.67,0.39,0.31


In [43]:
bleu_withoutr_sort = merged_df.sort_values(by=["bleu_without"])
bleu_withoutr_sort

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,letter_id,n_footnote,generated_footnote,xml_footnote,text_footnote,bert_with,bert_without,bleu_with,rouge_with,bleu_without,rouge_without
0,0,0,0,12891,10,"Die drei Briefe, die ich empfangen habe, sind von dem König von Frankreich, dem König von England und dem König von Spanien.",1mm von einandren gan: Beim Auseinandergehen (nach der Sitzung).,1mm von einandren gan: Beim Auseinandergehen (nach der Sitzung).,0.623506,0.623506,0.000000,0.058824,0.000000,0.058824
1898,1898,1898,1898,12071,6,"Über die Person von <persName ref=""p7147"" cert=""high"">Iacobo</persName>, die nicht mit Sicherheit identifiziert werden kann, aber möglicherweise mit <persName ref=""p7147"" cert=""high"">Jacob Sturm von Aich</persName> identisch ist.","<persName ref=""p7147"" cert=""high"">Jakob Würben</persName>, seit 1528 Verweser und seit 1529 Pfarrer und Seelsorger an der Leutkirche in <placeName ref=""l49"" cert=""high"">Biel</placeName>; s. J[acob] <hi>Wyß,</hi> Das <placeName ref=""l49"" cert=""high"">Bieler</placeName> Schulwesen, <placeName ref=""l49"" cert=""high"">Biel</placeName> 1919, S. 18.","Jakob Würben, seit 1528 Verweser und seit 1529 Pfarrer und Seelsorger an der Leutkirche in Biel; s. J[acob] Wyß, Das Bieler Schulwesen, Biel 1919, S. 18.",0.764247,0.600765,0.159344,0.204545,0.000000,0.000000
1899,1899,1899,1899,11535,5,"<bibl>Plautus, Mercator, 101</bibl>, vgl. <bibl>Adagia</bibl>, 1, 14, 44 (<bibl>LB</bibl> II 383).",Über das Gegengeschenk Bullingers ist nichts Weiteres bekannt.,Über das Gegengeschenk Bullingers ist nichts Weiteres bekannt.,0.535652,0.593493,0.000000,0.000000,0.000000,0.000000
1900,1900,1900,1900,11251,19,"Suspiciones me vexant contra cohabitatores nostros, quia de eorum vita et conversatione multa et varia rumoribus audiuntur.","<persName ref=""p18477"" cert=""high"">Ferdinand I.</persName>, römischer König.","Ferdinand I., römischer König.",0.496785,0.551276,0.000000,0.000000,0.000000,0.000000
1901,1901,1901,1901,11317,5,"Das ist nicht möglich, da die Fußnote 5 nicht existiert.","Das Werk erschien 1541 bei <persName ref=""p17826"" cert=""high"">Bartholomäus Westheimer</persName>; s. <bibl>VD 16, S 5248</bibl> (vgl. auch die unfirmierte Ausgabe S 5247).","Das Werk erschien 1541 bei Bartholomäus Westheimer; s. VD 16, S 5248 (vgl. auch die unfirmierte Ausgabe S 5247).",0.605980,0.649361,0.000000,0.100000,0.000000,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,862,862,862,13003,7,"Zum ital. Krieg, s. zuletzt <ref target=""file12992"">Nr. 2943</ref>,<ptr target=""qvzl17208-2"" type=""qv""/>.","Siehe dazu zuletzt <ref target=""file12992"">Nr. 2943</ref>,<ptr target=""qvzl17208-5"" type=""qv""/>.","Siehe dazu zuletzt Nr. 2943,.",0.926463,0.779939,0.757681,0.750000,0.380580,0.500000
2414,2414,2414,2414,13003,3,"Zu diesem s. zuletzt <ref target=""file12992"">Nr. 2943</ref>,<ptr target=""qvzl17208-2"" type=""qv""/>.","Bezug auf Myconius' Bitte in <ref target=""file12992"">Nr. 2943</ref>,<ptr target=""qvzl17208-4"" type=""qv""/>.","Bezug auf Myconius' Bitte in Nr. 2943,.",0.935838,0.762249,0.792523,0.666667,0.392815,0.307692
1002,1002,1002,1002,12198,17,"Zum Ausschluss der Ehebruchssünde des <persName ref=""p8130"" cert=""high"">Lorenz Meyers</persName> am <placeName ref=""l540"" cert=""high"">Wellenberg</placeName> s. <bibl>HBBW</bibl> XIV <ref target=""file11959"">[Nr. 1912]</ref> 236.","Der <placeName ref=""l540"" cert=""high"">Wellenberg</placeName>-Turm, Gefängnis der Stadt Zürich; s. <bibl>HBBW</bibl> XIV <ref target=""file11959"">[Nr. 1912]</ref> 236, Anm. 1.","Der Wellenberg-Turm, Gefängnis der Stadt Zürich; s. HBBW XIV [Nr. 1912] 236, Anm. 1.",0.877128,0.795429,0.621101,0.634921,0.411039,0.484848
1939,1939,1939,1939,11851,5,Der Wirttenbergische Herzog ist Herzog Ulrich von Württemberg.,"<persName ref=""p9780"" cert=""high"">Herzog Ulrich von Württemberg</persName>.",Herzog Ulrich von Württemberg.,0.622317,0.827372,0.074442,0.500000,0.446324,0.714286


In [26]:
merged_df.to_csv("evaluation-Meta-Llama-3.1-8B-Instruct_instruct_add_test.csv")

# Evaluate GPT generation

In [6]:
generated_footnotes_df = pd.read_csv("../../data/model_responses/gpt/gpt-4o-mini-2024-07-18-instruct_add-test.csv")
eval_df_filename = "evaluation-gpt-4o-mini-2024-07-18-instruct_add-test.csv"
merged_df = get_merged_df(human_footnotes_df, generated_footnotes_df, eval_df_filename)
evaluator = genEvaluator(merged_df)

bert_without is missing
No BLEU and ROUGE with markup are missing
No BLEU and ROUGE without are missing


In [7]:
evaluator.compute_bertscore_without_markup()

Computing BERT without Markup


  0%|          | 0/3239 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 3239/3239 [00:39<00:00, 82.75it/s]


In [8]:
merged_df = evaluator.update_and_return_df()

In [9]:
merged_df.to_csv(eval_df_filename)

In [12]:
bert_with_sort = merged_df.sort_values(by=["bert_with"])
bert_with_sort

,Unnamed: 0.1,Unnamed: 0,letter_id,n_footnote,generated_footnote,xml_footnote,text_footnote,bert_with,bert_without
3117,3117,3117,13043,4,"Fussnote 4 könnte wie folgt formuliert werden:\n\n""Die Relevanz der Buße in der reformatorischen Theologie ist grundlegend, da sie eine zentrale Rolle im Verständnis von Sünde und Erlösung spielt. Diese Aussage reflektiert die Überzeugung, dass nur durch aufrichtige Reue und Umkehr die Beziehung zu Gott wiederhergestellt werden kann. In dieser Zeit war die Diskussion um die Wiedereinführung oder Reform der Sakramente, insbesondere der Buße, ein scharf umstrittenes Thema im Rahmen der evangelischen Bewegung.""","Vgl. <ref target=""file12950"">Nr. 2901</ref>,<ptr target=""qvzl17167-2"" type=""qv""/>; <ref target=""file12983"">Nr. 2934</ref>,<ptr target=""qvzl17202-5"" type=""qv""/>. <ptr target=""qvzl17202-6"" type=""qv""/>; <ref target=""file13003"">Nr. 2954</ref>,<ptr target=""qvzl17220-10"" type=""qv""/>, und unten Z. <ptr target=""qvzl17260-2"" type=""qv""/>.","Vgl. Nr. 2901,; Nr. 2934,. ; Nr. 2954,, und unten Z. .",0.479952,0.522403
1195,1195,1195,11503,94,"In der Tatsache, dass im vorliegenden Kontext „redibition“ oft als ein Rückgaberecht und eine Form der Anfechtung bei fehlerhaften Käufen interpretiert wird, verweist die Geschichte des Rechts auf die Praxis des römischen Rechts, die besonders bei Kaufverträgen von Bedeutung war. Hierbei handelt es sich um die Möglichkeit für Käufer, die zurückgegebenen Waren nachzuweisen, dass diese Mängel aufwiesen oder in irgendeiner Weise fehlerhaft waren, was bei der Entstehung des Rechts auf Rückgabe entscheidend war.","<persName ref=""p18547"" cert=""high"">Ludwig II. der Deutsche</persName>, Sohn Kaiser <persName ref=""p18547"" cert=""high"">Ludwigs</persName> des Frommen.","Ludwig II. der Deutsche, Sohn Kaiser Ludwigs des Frommen.",0.480337,0.586311
182,182,182,10237,11,"In dieser Passage bezieht sich Haller auf die verschiedenen Streitfragen und Spannungen, die innerhalb der reformierten Gemeinschaften und zwischen diesen und den katholischen Autoritäten bestehen. Die ""quaestiones motas"" könnten sich auf theologische Differenzen beziehen, die zu Konflikten und Missverständnissen führen, sowie auf die Herausforderungen, mit denen die Reformatoren konfrontiert sind, während sie versuchen, die Einheit in der Lehre zu bewahren und gleichzeitig mit Widerstand und Anfeindungen umzugehen.",Siehe oben S. 62f. 83. 86f.,Siehe oben S. 62f. 83. 86f.,0.482067,0.482067
2768,2768,2768,12818,14,"„Rogo ergo denuo“ bedeutet in etwa „Ich bitte dich erneut“. Haller drückt hier seine Dringlichkeit aus, die Unterzeichnung eines Consilium senatus zu erwirken und so die Unsicherheit und die dringliche Lage in Augsburg zu klären. Er zeigt sich besorgt über die anhaltende Untätigkeit und die Möglichkeit, dass er, seine Familie und andere Angehörige seiner Gemeinde in Gefahr geraten könnten, wenn keine klare Entscheidung getroffen wird.","Siehe schon <ref target=""file12793"">Nr. 2744</ref>,56f; <ref target=""file12808"">Nr. 2759</ref>,79-86.","Siehe schon Nr. 2744,56f; Nr. 2759,79-86.",0.487417,0.520060
2171,2171,2171,12414,16,"Die in diesem Brief erwähnte „frater observandissime“ ist eine respektvolle Anrede für Heinrich Bullinger, die Frölichs Freundschaft und Wertschätzung ausdrückt. Diese Formulierung zeigt die enge Verbundenheit zwischen den Reformatoren und die Wichtigkeit von Freundschaften in ihrer kirchlichen und theologischen Arbeit während der Reformationszeit. Der Brief selbst ist ein Beispiel für die persönliche Korrespondenz unter den führenden Reformatoren und spiegelt die Schwierigkeiten und Herausforderungen wider, mit denen sie in dieser turbulenten Zeit konfrontiert waren, insbesondere hinsichtlich ihrer Gesundheitsprobleme und ihrer Bemühungen um die Kirche.",Nr. 2351.,Nr. 2351.,0.487529,0.487529
...,...,...,...,...,...,...,...,...,...
556,556,556,10668,3,"Bullingers Ehefrau <persName ref=""p7041"" cert=""high"">Margarethe Bullinger, geb. Hailb

# check the BLEU funcion

In [35]:
pred = "das hier gleich"
ref = "das hier gleich"
bleu.compute(predictions=[pred], references=[ref])["bleu"]

0.0

In [30]:
pred = "das hier gleich."
ref = "das hier gleich."
bleu.compute(predictions=[pred], references=[ref])["bleu"]

1.0

In [31]:
%pip install --upgrade bleu
from bleu import multi_list_bleu

  Preparing metadata (setup.py) ... done
  Created wheel for bleu: filename=bleu-0.3-py3-none-any.whl size=5780 sha256=2af356cf5f14ab8665c8892c467e11361d876e157a38c08cbda19767cdb89e66
  Stored in directory: /root/.cache/pip/wheels/c6/d8/d1/009abe01b8b2c6a14c62d197b510b3cc1076014c22d712c5ce
Successfully built bleu


In [38]:

pred = "das hier gleich."
ref = "das hier gleich."
multi_list_bleu([[ref]], [[pred]])

[Info] Starting to run this command now: perl /tmp/tmp_bleu/multi-bleu-detok.perl /tmp/tmp_bleu/ref_dtk0.txt < /tmp/tmp_bleu/hyp_dtk0.txt 


[100.0]

In [39]:
pred = "das hier gleich"
ref = "das hier gleich"
multi_list_bleu([[ref]], [[pred]])

[Info] Starting to run this command now: perl /tmp/tmp_bleu/multi-bleu-detok.perl /tmp/tmp_bleu/ref_dtk0.txt < /tmp/tmp_bleu/hyp_dtk0.txt 


[0.0]

Bleu is very strict... For example these simple ones, that are quite similar, bc they are all bible refs. But I guess they are too short?